# Download iEEG

In [16]:
from ieeg.auth import Session
import pandas as pd
import matlab.engine
import math
import os
from datetime import datetime, timedelta

from ieeg_utils import *

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import FuncFormatter

from scipy import stats, io
from scipy.fft import fft

import numpy as np
from numpy import pi, NaN

## Load data

In [2]:
patients_df = pd.read_excel("./data/selected_patients.xlsx")
patients_df

,hup_id,region,lateralization,weight_kg,r_id
0,141,mesial temporal,right,85.7,294
1,142,mesial temporal,left,65.3,295
2,181,mesial temporal,left,63.0,490
3,185,mesial temporal,left,76.2,332
4,187,mesial temporal,right,86.2,522


In [3]:
# Load master_elecs.csv from ./data/
master_elecs_df = pd.read_csv("./data/master_elecs.csv")

# only take the numbers in rid column
master_elecs_df["rid"] = master_elecs_df["rid"].str.extract("(\d+)", expand=False)
master_elecs_df["rid"] = master_elecs_df["rid"].astype(int)

# Drop mni_x, mni_y, mni_z, mm_x, mm_y, mm_z columns
master_elecs_df = master_elecs_df.drop(
    columns=["mni_x", "mni_y", "mni_z", "mm_x", "mm_y", "mm_z"]
)

master_elecs_df

,rid,name,vox_x,vox_y,vox_z,label,soz,resected,spike_rate,engel
0,13,LST01,80.6116,106.5480,64.5941,left inferior temporal,False,False,1.091902,1.0
1,13,LST02,72.0779,109.4150,63.1223,left inferior temporal,False,False,1.091902,1.0
2,13,LST03,64.9060,112.3760,68.7455,EmptyLabel,False,False,1.419472,1.0
3,13,LST04,65.0210,114.6600,78.2339,left middle temporal,False,False,0.655141,1.0
4,13,MST01,131.7410,64.3756,70.4205,right lingual,True,False,3.439490,1.0
...,...,...,...,...,...,...,...,...,...,...
14212,785,RB08,154.2550,114.2730,136.7560,EmptyLabel,False,NaN,0.369914,1.0
14213,785,RB09,159.1350,111.9920,136.6960,EmptyLabel,False,NaN,0.665845,1.0
14214,785,RB10,164.7520,109.9030,137.7640,right middle temporal,False,NaN,4.586930,1.0
14215,785,RB11,169.6320,107.6220,137.7040,right middle temporal,False,NaN,2.071517,1.0


In [4]:
# Load rid_hup_table.csv from ./data/
rid_hup_table_df = pd.read_csv("./data/rid_hup_table.csv")
# Drop the t3_subject_id and ieegportalsubjno columns
rid_hup_table_df = rid_hup_table_df.drop(columns=["t3_subject_id", "ieegportalsubjno"])
rid_hup_table_df

,record_id,hupsubjno
0,623,35
1,624,36
2,625,37
3,626,38
4,627,39
...,...,...
212,534,250
213,923,251
214,918,252
215,864,253


In [5]:
# Load aed_ref_ranges.xlsx from ./data/
aed_ref_ranges_df = pd.read_excel("./data/aed_ref_ranges.xlsx")
# Lowercase Drug column
aed_ref_ranges_df["Drug"] = aed_ref_ranges_df["Drug"].str.lower()
# show unique units
# print(aed_ref_ranges_df["Unit"].unique())
# mg/L and ug/mL are the same
# If Unit is ng/mL, convert to ug/mL
aed_ref_ranges_df.loc[aed_ref_ranges_df["Unit"] == "ng/mL", "Min"] = (
    aed_ref_ranges_df["Min"] / 1000
)
aed_ref_ranges_df.loc[aed_ref_ranges_df["Unit"] == "ng/mL", "Max"] = (
    aed_ref_ranges_df["Max"] / 1000
)
# Add a column that takes the average of Min and Max
aed_ref_ranges_df["Avg"] = (aed_ref_ranges_df["Min"] + aed_ref_ranges_df["Max"]) / 2
aed_ref_ranges_df

,Drug,Min,Max,Unit,Avg
0,levetiracetam,12.00,46.00,mg/L,29.000
1,carbamazepine,4.00,10.00,mg/L,7.000
2,oxcarbazepine,3.00,35.00,ug/mL,19.000
3,clobazam,0.03,0.30,ng/mL,0.165
4,n-desmethylclobazam,0.30,3.00,ng/mL,1.650
5,topiramate,5.00,20.00,mg/L,12.500
6,valproic acid,50.00,125.00,ug/mL,87.500
7,lacosamide,1.00,10.00,ug/mL,5.500
8,felbamate,30.00,60.00,ug/mL,45.000
9,lamotrigine,2.50,15.00,mg/L,8.750


In [6]:
patient_hup_ids = patients_df.hup_id.to_numpy()
patient_weights = patients_df.weight_kg.to_numpy()
assert len(patient_hup_ids) == len(patient_weights)
len(patient_hup_ids)

5

In [7]:
all_patient_hup_ids = pd.read_excel("./data/HUP_implant_dates.xlsx")
all_patient_hup_ids = all_patient_hup_ids["ptID"].to_numpy()
all_patient_hup_ids

array([225, 224, 223, 221, 219, 217, 216, 215, 214, 213, 211, 210, 209,
       208, 207, 206, 205, 204, 202, 201, 199, 197, 196, 195, 194, 193,
       192, 191, 190, 189, 188, 187, 186, 185, 184, 182, 181, 180, 179,
       178, 177, 175, 174, 173, 172, 171, 170, 169, 168, 167, 166, 165,
       164, 163, 162, 161, 160, 159, 158, 157, 156, 155, 154, 153, 152,
       151, 150, 149, 148, 147, 146, 145, 144, 143, 142, 141, 140, 139,
       138, 137])

In [8]:
# Create a mapping between patient ids and the index of the patient in the patients_df dataframe
patient_hup_id_to_index = {}
for i, patient_id in enumerate(all_patient_hup_ids):
    patient_hup_id_to_index[patient_id] = i
patient_hup_id_to_index

{225: 0,
 224: 1,
 223: 2,
 221: 3,
 219: 4,
 217: 5,
 216: 6,
 215: 7,
 214: 8,
 213: 9,
 211: 10,
 210: 11,
 209: 12,
 208: 13,
 207: 14,
 206: 15,
 205: 16,
 204: 17,
 202: 18,
 201: 19,
 199: 20,
 197: 21,
 196: 22,
 195: 23,
 194: 24,
 193: 25,
 192: 26,
 191: 27,
 190: 28,
 189: 29,
 188: 30,
 187: 31,
 186: 32,
 185: 33,
 184: 34,
 182: 35,
 181: 36,
 180: 37,
 179: 38,
 178: 39,
 177: 40,
 175: 41,
 174: 42,
 173: 43,
 172: 44,
 171: 45,
 170: 46,
 169: 47,
 168: 48,
 167: 49,
 166: 50,
 165: 51,
 164: 52,
 163: 53,
 162: 54,
 161: 55,
 160: 56,
 159: 57,
 158: 58,
 157: 59,
 156: 60,
 155: 61,
 154: 62,
 153: 63,
 152: 64,
 151: 65,
 150: 66,
 149: 67,
 148: 68,
 147: 69,
 146: 70,
 145: 71,
 144: 72,
 143: 73,
 142: 74,
 141: 75,
 140: 76,
 139: 77,
 138: 78,
 137: 79}

In [10]:
ieeg_offset_row1_df = pd.read_excel("data/ieeg_offset/row_1.xlsx", header=None)
ieeg_offset_row2_df = pd.read_excel("data/ieeg_offset/row_2.xlsx", header=None)
ieeg_offset_row3_df = pd.read_excel("data/ieeg_offset/row_3.xlsx", header=None)

In [19]:
eng = matlab.engine.start_matlab()
nina_path = eng.genpath("/Volumes/USERS/nghosn3/Pioneer/spikes-AED/aed_dose_modeling")
eng.addpath(nina_path, nargout=0)

In [20]:
get_aed_curve_kg_res = eng.get_aed_curve_kg(patient_hup_ids, patient_weights, nargout=4)

HUP141
HUP141 has medications with NaN dose: 
HUP142
HUP181
HUP185
HUP185 has medications with NaN dose: 
HUP187
HUP187 has medications with NaN dose: 


In [21]:
dose_curves_res = get_aed_curve_kg_res[0]
tHr_res = get_aed_curve_kg_res[1]
med_names_res = get_aed_curve_kg_res[3]
assert (
    len(dose_curves_res) == len(tHr_res) == len(med_names_res) == len(patient_hup_ids)
)
print(len(dose_curves_res), len(tHr_res), len(med_names_res), len(patient_hup_ids))

5 5 5 5


In [22]:
devin_path = eng.genpath("./matlab")
eng.addpath(devin_path, nargout=0)

In [ ]:
patient_idx = []
for i, patient_id in enumerate(all_patient_hup_ids):
    if patient_id in patient_hup_ids:
        patient_idx.append(i)
patient_idx

ieeg_starts = []
for i in patient_idx:
    ieeg_starts.append(eng.get_ieeg_start(i + 1, nargout=1))

ieeg_starts = np.array(ieeg_starts).flatten()
ieeg_starts

# Save ieeg_starts to a npy file
np.save("./data/ieeg_starts_3.npy", ieeg_starts)

In [13]:
ieeg_starts = np.load("./data/ieeg_starts_3.npy")
ieeg_starts

array([33.71638889, 37.78333333, 35.30527778, 35.65194444, 37.24944444])

## Download

In [23]:
def create_pwd_file(username, password, fname=None):
    if fname is None:
        fname = "{}_ieeglogin.bin".format(username[:3])
    with open(fname, "wb") as f:
        f.write(password.encode())
    print("-- -- IEEG password file saved -- --")


create_pwd_file("dma", "mycqEv-pevfo4-roqfan")

with open("dma_ieeglogin.bin", "r") as f:
    session = Session("dma", f.read())

-- -- IEEG password file saved -- --


In [25]:
for i, row in patients_df.iterrows():
    # Get patient id and weight
    patient_hup_id, patient_weight = row.hup_id, row.weight_kg
    patient_idx = patient_hup_id_to_index[patient_hup_id]

    #########################################
    # Run matlab script
    #########################################
    aligned_emu_start_time_hrs = round(ieeg_starts[i])
    seizure_times = np.array(
        eng.get_seizure_times(f"HUP{patient_hup_id}", nargout=2)[0]
    )

    #########################################
    # Check tables
    #########################################

    if rid_hup_table_df.loc[rid_hup_table_df["hupsubjno"] == patient_hup_id].empty:
        print(f"HUP{patient_hup_id} has no info on rid_hup_table sheet")
        continue

    patient_rid = int(
        rid_hup_table_df.loc[rid_hup_table_df["hupsubjno"] == patient_hup_id].record_id
    )

    print(f"HUP{patient_hup_id} has rid {patient_rid}")

    patient_electrodes_df = master_elecs_df.loc[master_elecs_df["rid"] == patient_rid]

    if patient_electrodes_df.empty:
        print(f"HUP{patient_hup_id} has no info on master electrode sheet")
        continue

    #########################################
    # Get ieeg dataset name
    #########################################

    ieeg_dataset_names = []
    # Check if ieeg_offset_row1_df[patient_idx] is all NaNs
    if ieeg_offset_row1_df[patient_idx].isnull().values.all():
        ieeg_dataset_names = [f"HUP{patient_hup_id}_phaseII"]
    else:
        ieeg_dataset_names = ieeg_offset_row1_df[patient_idx].dropna().to_list()

    print(f"HUP{patient_hup_id} has {str(ieeg_dataset_names)} datasets")

    #########################################
    # Get the useful channels
    #########################################

    dataset_name = ieeg_dataset_names[0]
    ds = session.open_dataset(dataset_name)
    channel_labels = ds.get_channel_labels()
    channel_types_df = check_channel_types(channel_labels)

    seeg_electrodes_df = channel_types_df.loc[
        (channel_types_df["type"] == "seeg") | (channel_types_df["type"] == "ecog")
    ]

    # Find the rows in patient_electrodes_df that correspond to seeg_electrodes
    seeg_electrodes_df = patient_electrodes_df.loc[
        patient_electrodes_df["name"].isin(seeg_electrodes_df.name.to_numpy(dtype=str))
    ]
    # Find the ones that have label not 'EmptyLabel'
    grey_matter_seeg_electrodes_df = seeg_electrodes_df.loc[
        seeg_electrodes_df["label"] != "EmptyLabel"
    ]

    # Find the indices of the names of the grey matter electrodes in channel_types_df
    grey_matter_seeg_electrodes_indices = channel_types_df.loc[
        channel_types_df["name"].isin(grey_matter_seeg_electrodes_df.name)
    ].copy()  # Added .copy() here to create a new DataFrame

    all_indices = grey_matter_seeg_electrodes_indices.name.to_numpy(dtype=str)

    #########################################
    # Get AED dose
    #########################################

    # Assign results to variables
    all_dose_curves = dose_curves_res[i]
    all_tHr = tHr_res[i]
    all_med_names = med_names_res[i]

    print(all_med_names)

    assert (
        len(all_dose_curves) == len(all_tHr) == len(all_med_names)
    ), "all_dose_curves, all_tHr, and all_med_names should have the same length"

    # Process numpy arrays
    all_med_names_plot = np.array(all_med_names).flatten()

    tmp = []
    for dose in all_dose_curves:
        dose = np.array(dose).flatten()
        tmp.append(dose)
    all_dose_curves_plot = np.array(tmp, dtype=object)

    tmp = []
    for dose in all_tHr:
        dose = np.array(dose).flatten()
        tmp.append(dose)
    all_tHr_plot = np.array(tmp, dtype=object)

    # Get the last element of each array in all_tHr_plot and test if they're the same
    # They should be the same because they're the same patient
    assert all(
        [all_tHr_plot[i][-1] == all_tHr_plot[0][-1] for i in range(len(all_tHr_plot))]
    ), "The last element of each array in all_tHr_plot should be the same"

    # Construct the time axis
    emu_start_time_hrs = min([all_tHr_plot[i][0] for i in range(len(all_tHr_plot))])
    emu_end_time_hrs = all_tHr_plot[0][-1]
    max_dose_duration_hrs = emu_end_time_hrs - emu_start_time_hrs
    max_length = max([len(all_tHr_plot[i]) for i in range(len(all_tHr_plot))])
    time_axis = np.linspace(emu_start_time_hrs, emu_end_time_hrs, max_length)
    print(aligned_emu_start_time_hrs)
    # Print start and end times in one line
    print(
        f"Doses: Start time: {emu_start_time_hrs} hr, End time: {emu_end_time_hrs} hr"
    )

    #########################################
    # Get iEEG data
    #########################################

    for dataset_idx, dataset_name in enumerate(ieeg_dataset_names):
        eeg_offset_sec = ieeg_offset_row2_df[patient_idx][dataset_idx]
        eeg_offset_usec, eeg_offset_hrs = eeg_offset_sec * 1e6, int(
            eeg_offset_sec / 3600
        )
        assert (
            eeg_offset_sec > 0
        ), "ieeg_offset_sec should be positive, i.e. iEEG starts after AEDs"

        print(f"Opening {dataset_name}, it has offset of {eeg_offset_sec} seconds")
        # check if aligned_emu_start_time_hrs is an integer

        hourly_eeg = []
        eeg_end_time_hrs = None

        for hour in range(int(max_dose_duration_hrs) + 1):
            print(f"Getting iEEG data for hour {hour}")
            duration_usec = 6e8  # 10 minute
            start_time_usec = hour * 3600 * 1e6
            stop_time_usec = start_time_usec + duration_usec

            try:
                local_res, sample_rate = get_iEEG_data(
                    "dma",
                    "dma_ieeglogin.bin",
                    dataset_name,
                    start_time_usec,
                    stop_time_usec,
                    select_electrodes=all_indices,
                )
            except Exception as e:
                # handle the exception
                break

            assert local_res.shape[1] == len(grey_matter_seeg_electrodes_indices)

            # Drop rows that are all NaN
            local_res = local_res.dropna(axis=0, how="any")
            # Check if local_res is empty
            if local_res.empty:
                eeg_end_time_hrs = hour
                break
            else:
                # Try Save local_res to a pickle file
                with open(
                    f"./data/10min/HUP_{patient_hup_id}_hr_{hour}_fs_{int(sample_rate)}.pkl",
                    "wb",
                ) as f:
                    pickle.dump(local_res, f)

HUP141 has rid 294
HUP141 has ['HUP141_phaseII'] datasets
['lamotrigine', 'levetiracetam', 'lorazepam', 'oxcarbazepine']
34
Doses: Start time: 18.383333333333333 hr, End time: 194.78333333333333 hr
Opening HUP141_phaseII, it has offset of 133760.0 seconds
Getting iEEG data for hour 0
Getting iEEG data for hour 1
Getting iEEG data for hour 2
Getting iEEG data for hour 3
Getting iEEG data for hour 4
Getting iEEG data for hour 5
Getting iEEG data for hour 6
Getting iEEG data for hour 7
Getting iEEG data for hour 8
Getting iEEG data for hour 9
Getting iEEG data for hour 10
Getting iEEG data for hour 11
Getting iEEG data for hour 12
Getting iEEG data for hour 13
Getting iEEG data for hour 14
Getting iEEG data for hour 15
Getting iEEG data for hour 16
Getting iEEG data for hour 17
Getting iEEG data for hour 18
Getting iEEG data for hour 19
Getting iEEG data for hour 20
Getting iEEG data for hour 21
Getting iEEG data for hour 22
Getting iEEG data for hour 23
Getting iEEG data for hour 24
Gett